# Load Modules

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
os.environ['DISPLAY'] = ':999'
sys.path.append('/home/zachpen87/clearmap/ClearMap2/')

import h5py
import holoviews as hv
import dask.array as da
import scipy.ndimage as ndimage
import skimage
from ClearMap.Environment import * 
import ClearMap.ImageProcessing.H5 as H5img
hv.notebook_extension('bokeh')

# Specify List of Directories

In [ ]:
dir_list = [
    '/home/zachpen87/clearmap/data/SEFL17/ms_01/',
    '/home/zachpen87/clearmap/data/SEFL17/ms_02/',
    '/home/zachpen87/clearmap/data/SEFL17/ms_03/',
    '/home/zachpen87/clearmap/data/SEFL17/ms_04/',
    '/home/zachpen87/clearmap/data/SEFL17/ms_05/',
    '/home/zachpen87/clearmap/data/SEFL17/ms_06/',
    '/home/zachpen87/clearmap/data/SEFL17/ms_07/',
    '/home/zachpen87/clearmap/data/SEFL17/ms_08/',
    '/home/zachpen87/clearmap/data/SEFL17/ms_09/',
    '/home/zachpen87/clearmap/data/SEFL17/ms_10/',
    '/home/zachpen87/clearmap/data/SEFL17/ms_11/',
    '/home/zachpen87/clearmap/data/SEFL17/ms_12/',
    '/home/zachpen87/clearmap/data/SEFL17/ms_13/',
    '/home/zachpen87/clearmap/data/SEFL17/ms_14/',
    '/home/zachpen87/clearmap/data/SEFL17/ms_15/',
    '/home/zachpen87/clearmap/data/SEFL17/ms_16/',
    '/home/zachpen87/clearmap/data/SEFL17/ms_17/',
    '/home/zachpen87/clearmap/data/SEFL17/ms_18/',
    '/home/zachpen87/clearmap/data/SEFL17/ms_19/',
    '/home/zachpen87/clearmap/data/SEFL17/ms_20/',
    '/home/zachpen87/clearmap/data/SEFL17/ms_21/',
    '/home/zachpen87/clearmap/data/SEFL17/ms_22/',
    '/home/zachpen87/clearmap/data/SEFL17/ms_23/',
    '/home/zachpen87/clearmap/data/SEFL17/ms_24/',
    '/home/zachpen87/clearmap/data/SEFL17/ms_25/',
    '/home/zachpen87/clearmap/data/SEFL17/ms_26/',
    '/home/zachpen87/clearmap/data/SEFL17/ms_27/',
    '/home/zachpen87/clearmap/data/SEFL17/ms_28/',
]

# Define shape of heatmap

In [ ]:
ABA_directory = '/home/zachpen87/clearmap/AtlasDocs/Horizontal'
ABA_ref = 'ABA_25um_reference__1_2_3__slice_None_None_None__slice_None_None_None__slice_None_None_None__.tif'
ref = skimage.io.imread(os.path.join(ABA_directory, ABA_ref))
htmp_shape = ref.shape
del ref

# Main Loop

In [ ]:
for directory in dir_list:

    # load coordinate files and slice info
    hdf5_file   = os.path.join(directory,'data.hdf5')
    coords_file = os.path.join(directory,'coords.npy')
    coords = np.load(coords_file)
    slc = (slice(None,None), slice(coords[0,0],coords[0,1]), slice(coords[1,0],coords[1,1]))
    
    
    # load workspace
    ws = wsp.Workspace('CellMap', directory=directory);
    ws.update(raw='fos.npy', autofluorescence='auto.npy')
    resources_directory = settings.resources_path
    
    
    # print hdf5 dataset, workspace, and coordinate info
    with h5py.File(hdf5_file,'a') as f:
        print('hdf5_file datasets: {x}'.format(x=list(f.keys())))
        print('hdf5_file shape: {x}\n'.format(x=f['fos'].shape))
    print(ws.info())
    print('slice coordinates: y={y}, x={x}'.format(y=coords[0],x=coords[1]))
    
    
    #smooth image
    H5img.hdf5_filter(
        hdf5_file = hdf5_file,
        dset_in = 'fos',
        dset_out = 'smooth',
        filt = 'median', ksize  = (3,3,3),
        slc = slc)
    
    
    #remove background
    H5img.hdf5_filter(
        hdf5_file = hdf5_file,
        dset_in = 'smooth',
        dset_out = 'bg',
        filt = 'opening', ksize  = (5,5,5))
    H5img.hdf5_bgsubtract(
        hdf5_file = hdf5_file,
        dset_img = 'smooth',
        dset_bg = 'bg',
        dset_out = 'bg_rmv')
    
    
    #threshold background subtracted image
    threshold = 8
    with h5py.File(hdf5_file,'a') as f:
        image = da.from_array(
                f['bg_rmv'],
                chunks = (400,400,400)) 
        image[image<threshold] = 0
        image.to_hdf5(hdf5_file,('/'+'thresh'))
        
    
    #calculate distance transform
    H5img.hdf5_filter(
        hdf5_file = hdf5_file,
        dset_in = 'thresh',
        dset_out = 'dist',
        filt = 'distance_transform')
    
    
    #define local maxima of distance transform
    H5img.hdf5_filter(
        hdf5_file = hdf5_file,
        dset_in = 'dist',
        dset_out = 'local_max',
        filt = 'local_max',
        ksize = (10,10,10))
        
    
    #label continuous local maxima
    H5img.label(
        hdf5_file = hdf5_file,
        dset_in = 'local_max',
        dset_out = 'lbls',
        min_size = None, max_size=500,
        chunk_dimension = 1,
        chunk_size = 350,
        chunk_overlap = 25)
   

    #find centroids and save
    centroids = H5img.find_centroids(
        hdf5_file = hdf5_file,
        dset_lbls = 'lbls',
        dset_wts = 'bg_rmv',
        chunk_dimension = 1,
        chunk_size = 400,
        chunk_overlap = 25)
    with h5py.File(hdf5_file,'a') as f:
        centroids = centroids.astype('int')
        if 'centroids/original' in f.keys():
            del f['centroids/original']
        f.create_dataset('centroids/original', data=centroids)
        
   
    #transform centroids and get labels
    with h5py.File(hdf5_file,'a') as f:
        centroids = f['centroids/original'][:]
        input_shape = f['bg_rmv'].shape

        coordinates = res.resample_points(
                          np.flip(centroids,axis=1), sink=None, orientation=None, 
                          source_shape = tuple(np.flip(input_shape)), 
                          sink_shape = io.shape(ws.filename('resampled')));

        coordinates = elx.transform_points(
                          coordinates, sink=None, 
                          transform_directory=ws.filename('resampled_to_auto'), 
                          binary=True, indices=False);

        coordinates = elx.transform_points(
                          coordinates, sink=None, 
                          transform_directory=ws.filename('auto_to_reference'),
                          binary=True, indices=False);
        
        labels = ano.label_points(
            points =  coordinates,
            annotation_file = '/home/zachpen87/clearmap/AtlasDocs/Horizontal/ABA_25um_annotation.tif',
            key = 'id')

        coordinates = np.flip(coordinates, axis=1)
        if 'centroids/transformed' in f.keys():
            del f['centroids/transformed']
        if 'centroids/labels' in f.keys():
            del f['centroids/labels']
        f.create_dataset('centroids/transformed', data=coordinates)
        f.create_dataset('centroids/labels', data=labels)
    
    
    #create heatmap
    with h5py.File(hdf5_file,'a') as f:
        heatmap = np.zeros(htmp_shape)
        for x in coordinates.astype('int'):
            try:
                heatmap[x[0],x[1],x[2]] += 1
            except:
                pass
        heatmap = H5img.array_filter(heatmap,filt='gaussian',ksize=(2,2,2))
        if 'heatmap' in f.keys():
            del f['heatmap']
        f.create_dataset('heatmap', data = heatmap)
        
    
    # delete intermediate files
    temp_file = os.path.join(directory,'data_temp.hdf5')
    os.rename(hdf5_file, temp_file)
    with h5py.File(temp_file,'r') as tempf, h5py.File(hdf5_file,'w') as f:
        for key in ['fos','auto','bg_rmv','heatmap','centroids/original','centroids/transformed']:
            print('writing: {key}'.format(key=key))
            f.create_dataset(key, data=tempf[key][:], compression='lzf')
    os.remove(temp_file)